# Training eines Machine Learning Models
In diesem Jupyter Notebook werden drei Modelle trainiert, die die Position innerhalb der Fachhochschule Wedel anhand von Bildern klassifizieren

Das Training wurde auf Google Colab durchgeführt, daher können Pfadanpassungen für die Ausführung notwendig sein.

Imports und Konstanten 

In [ ]:
# Imports needed
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.utils import to_categorical

img_height = 320
img_width = 180
num_epochs = 15 # 15

batch_size = 15


nClasses_1 = 108
nClasses_2 = 143
nClasses_3 = 104

input_shape = (img_height, img_width, 1) #3 for rgb
SRC_DIR_1 = "/content/training1"
SRC_DIR_2 = "/content/training2"
SRC_DIR_3 = "/content/training3"

Hierdurch wird eine Verbindung zu Google Drive hergestellt, um die Trainingsdaten zu erreichen. 
Diese müssen für jede Sitzung hochgeladen werden. Der Datensatz wird aus Google Drive geladen und in das lokale Filesystem von Colab entpackt.

Zudem werden die Ordner angelegt, in die die Modelle später gespeichert werden.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!mkdir /content/gdrive/MyDrive/Thesis_ColabNotebooks/models/m1
!mkdir /content/gdrive/MyDrive/Thesis_ColabNotebooks/models/m2
!mkdir /content/gdrive/MyDrive/Thesis_ColabNotebooks/models/m3

Die Trainingsdaten werden aus Google Drive heraus entpackt und in das Filesystem von Colab selbst geladen, um ein möglichst schnelles Training zu ermöglichen.

In [ ]:
!unzip /content/gdrive/MyDrive/Thesis_ColabNotebooks/TrainData/ABC.zip -d /content/training1
!unzip /content/gdrive/MyDrive/Thesis_ColabNotebooks/TrainData/D.zip -d /content/training2
!unzip /content/gdrive/MyDrive/Thesis_ColabNotebooks/TrainData/EF.zip -d /content/training3

Im nächsten Abschnitt wird eine GPU auf Colab gesucht. Damit kann ein Model auf der GPU trainiert werden statt auf der CPU.

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Preprocess nimmt die Bilder und splittet sie in Trainins- und Testdaten. Dabei werden die Bilder jeweils gelabelt und zwar abhängig des Ordners, in dem sie sich befinden. Ein Bild in Ordner A0012 bekommt dementsprechend das Label A0012. Das Label steht für Gebäudeteil A, Stockwerk 0, an Position 01 im Gang und Blickrichtung 2. 

In [ ]:
def preprocess(source_dir):
    ds_train = tf.keras.preprocessing.image_dataset_from_directory(
        source_dir,
        labels="inferred",
        label_mode="categorical",  #int categorical, binary
        color_mode="grayscale",
        batch_size=batch_size,
        image_size=(img_height, img_width),  # reshape if not in this size
        shuffle=True,
        seed=123,
        validation_split=0.1,
        subset="training",
    )

    ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
        source_dir,
        labels="inferred",
        label_mode="categorical",  # categorical, binary
        color_mode="grayscale",
        batch_size=batch_size,
        image_size=(img_height, img_width),  # reshape if not in this size
        shuffle=True,
        seed=123,
        validation_split=0.1,
        subset="validation",
    )

    return (ds_train, ds_validation)

Hier wird die interne Struktur des Models definiert, indem nacheinander Convolutional Layer, Max Pooling Layer, Dropout Layer sowie Dense Layer eingefügt werden.

In [ ]:
def create_model(nClasses):
    
    model = Sequential()
    # The first two layers with 32 filters of window size 3x3
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25)) #prevents overfitting

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nClasses, activation='softmax'))

    return model

Mit dieser Funktion wird ein Model kompiliert, die interne Struktur ausgegeben, trainiert und anschließend im übergebenen Ordner folder_in_drive gespeichert.

In [ ]:
def train_model(model, ds_train, ds_validation, folder_in_drive):
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    model.summary()

    model.fit(ds_train, epochs=num_epochs, verbose=2)
    model.save('/content/gdrive/MyDrive/Thesis_ColabNotebooks/models/' + folder_in_drive)
    model.evaluate(ds_validation, verbose=2)

Vorbereitung der 3 Modelle

In [ ]:
(ds_train_m1, ds_validation_m1) = preprocess(SRC_DIR_1)
(ds_train_m2, ds_validation_m2) = preprocess(SRC_DIR_2)
(ds_train_m3, ds_validation_m3) = preprocess(SRC_DIR_3)

model1 = create_model(nClasses_1)
model2 = create_model(nClasses_2)
model3 = create_model(nClasses_3)


Training Model 1 (Gebäudeteil A, B und C)

In [ ]:
train_model(model1, ds_train_m1, ds_validation_m1, 'm1')

Training Model 2 (Gebäudeteil D)

In [ ]:
train_model(model2, ds_train_m2, ds_validation_m2, 'm2')

Training Model 3 (Gebäudeteil E und F)

In [ ]:
train_model(model3, ds_train_m3, ds_validation_m3, 'm3')